<a href="https://colab.research.google.com/github/gptix/data-visualization/blob/master/Copy_of_Copy_of_Copy_of_Data_Storytelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Visualization Project
- Subject: Health and evnironmental benefits of replacing U.S. generation of electrical power with nuclear generation

- Data sets
  - Power plants in the United States

##Power plants in the United States
###Munging before upload


Form EIA-923 detailed data with previous form data (EIA-906/920) 
at
https://www.eia.gov/electricity/data/eia923/

- Downloaded https://www.eia.gov/electricity/data/eia923/archive/xls/f923_2018.zip

- Unzipped

- Used file
EIA923_Schedules_2_3_4_5_M_12_2018_Final_Revision.xslx

- Saved sheet "Page 1 Generation and Fuel Data" as 
Page_1_Generation_and_Fuel_Data_2018.csv

- Saved copy as "working.csv"

- Deleted rows 1-5, not data.

- Replaced spaces with "_" in column name fields

- Replaced newlines with "_"

- Removed columns J, Q, R, with heading "Reserved", because the columns are empty and the column names are not unique.

- Pushed file to a github repository.

Repo: 

###Upload file to Drive



  
  - (extract coal power plants)
  - health effects of coal generation
  - Pollutants from coal
  - Model of terrestrial Energy power plant
  - Amount of fuel available for Terrestrial Energy
  - Environmental effects of nuclear power
  - deployability of Terrestrial Power to all sites
  - Modeled price per kwh for terrestrial



  Other Notes:
  Stopping use of coal will have benefits beyond cutting pollution. Tansportation of coal will be unnecessary. Mining will be unnecesary
  Disposal of waste will be unnececsaayr.


## Import code I might need.

In [0]:
# Import libraries probably needed.
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy import array
from numpy import linalg as la

from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO
import os.path

import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from functools import reduce
from math import sqrt

import random
from statistics import mean, stdev

In [0]:
# Load dataframe from CSV file stored on github.
url = "https://raw.githubusercontent.com/gptix/data-visualization/master/EIA923_Schedules_2_3_4_5_M_12_2018_Final_Revision.csv"
base_df = pd.read_csv(url)

# make a bckup of the dataframe in case mistakes are made during development.
base_df_bak = base_df
# Deprecated.
# base_df = pd.read_csv("https://raw.githubusercontent.com/gptix/data-visualization/master/working.csv")
base_df.shape

(13962, 94)

In [0]:
# Save metadata useful in mungung code.
base_df_cols = base_df.columns
# base_df_cols
# strings used in data file to represent coal fuel types.
# coal_fuel_types = ["ANT", "BIT", "LIG", "RC", "SC", "SGC", "SUB"]
AER_Fuel_Type_Code = 'COL'

In [0]:
# filter dataframe for only coal-fueled lines.
base_df = base_df[base_df.AER_Fuel_Type_Code.isin([AER_Fuel_Type_Code])]
base_df.shape

(758, 94)

In [0]:
# The data can be split into at least three tables;
# - power plants: each a site with one or more 'prime movers'
# - prime movers: each a machine that uses a particular fuel to generate power.
# - observations: monthly records of paraameter values for each prime mover.

# Create a separate dataframe to hold detailed data about power plants, keeping
# an index value to link to other dataframes.
plant_cols = ['Plant_Id', 'Combined_Heat_And_Power_Plant', 'Nuclear_Unit_Id', 
               'Plant_Name', 'Operator_Name', 'Operator_Id', 'Plant_State', 
               'Census_Region', 'NERC_Region', 'NAICS_Code', 
               'EIA_Sector_Number', 'Sector_Name']
drop_cols = list(set(base_df_cols) - set(plant_cols))
plant_detail = base_df.drop(columns=drop_cols)



prime_mover_cols = ['Plant_Id', 'Reported_Prime_Mover', 'Reported_Fuel_Type_Code',
       'AER_Fuel_Type_Code', 'Physical_Unit_Label']

other_unneeded_cols = ['Total_Fuel_Consumption_Quantity',
       'Electric_Fuel_Consumption_Quantity', 'Total_Fuel_Consumption_MMBtu',
       'Elec_Fuel_Consumption_MMBtu', 'Net_Generation_(Megawatthours)',
       'YEAR']

drop_cols2 = list(set(base_df_cols) - set(prime_mover_cols))

prime_mover_detail = base_df.drop(columns=drop_cols2)

drop_cols3 = ((set(prime_mover_cols) | 
              set(plant_cols) |
              set(other_unneeded_cols))
             - set(['Plant_Id']))

observations = base_df.drop(columns=drop_cols3)

observations.head()

,Plant_Id,Quantity_January,Quantity_February,Quantity_March,Quantity_April,Quantity_May,Quantity_June,Quantity_July,Quantity_August,Quantity_September,Quantity_October,Quantity_November,Quantity_December,Elec_Quantity_January,Elec_Quantity_February,Elec_Quantity_March,Elec_Quantity_April,Elec_Quantity_May,Elec_Quantity_June,Elec_Quantity_July,Elec_Quantity_August,Elec_Quantity_September,Elec_Quantity_October,Elec_Quantity_November,Elec_Quantity_December,MMBtuPer_Unit_January,MMBtuPer_Unit_February,MMBtuPer_Unit_March,MMBtuPer_Unit_April,MMBtuPer_Unit_May,MMBtuPer_Unit_June,MMBtuPer_Unit_July,MMBtuPer_Unit_August,MMBtuPer_Unit_September,MMBtuPer_Unit_October,MMBtuPer_Unit_November,MMBtuPer_Unit_December,Tot_MMBtu_January,Tot_MMBtu_February,Tot_MMBtu_March,Tot_MMBtu_April,Tot_MMBtu_May,Tot_MMBtu_June,Tot_MMBtu_July,Tot_MMBtu_August,Tot_MMBtu_September,Tot_MMBtu_October,Tot_MMBtu_November,Tot_MMBtu_December,Elec_MMBtu_January,Elec_MMBtu_February,Elec_MMBtu_March,Elec_MMBtu_April,Elec_MMBtu_May,Elec_MMBtu_June,Elec_MMBtu_July,Elec_MMBtu_August,Elec_MMBtu_September,Elec_MMBtu_October,Elec_MMBtu_November,Elec_MMBtu_December,Netgen_January,Netgen_February,Netgen_March,Netgen_April,Netgen_May,Netgen_June,Netgen_July,Netgen_August,Netgen_September,Netgen_October,Netgen_November,Netgen_December
3,3,.,0,.,.,.,.,.,.,.,.,.,.,.,0,.,.,.,.,.,.,.,.,.,.,.,0,.,.,.,.,.,.,.,.,.,.,.,0,.,.,.,.,.,.,.,.,.,.,.,0,.,.,.,.,.,.,.,.,.,.,.,0,.,.,.,.,.,.,.,.,.,.
4,3,227744,167125,54589,142706,.,246390,245048,230917,200588,21927,.,63906,227744,167125,54589,142706,.,246390,245048,230917,200588,21927,.,63906,20.95,20.46,20.65,20.58,.,20.3,20.29,20.67,20.45,20.39,.,19.97,4770781,3419378,1126990,2936604,.,5001471,4970799,4772593,4102827,447004,.,1275883,4770781,3419378,1126990,2936604,.,5001471,4970799,4772593,4102827,447004,.,1275883,474858,344963,111909,294779,.,500527,494818,468582,391806,34697,.,110414
6,3,0,.,151281,78674,.,0,0,0,0,0,.,0,0,.,151281,78674,.,0,0,0,0,0,.,0,0,.,17.77,17.72,.,0,0,0,0,0,.,0,0,.,2688566,1394418,.,0,0,0,0,0,.,0,0,.,2688566,1394418,.,0,0,0,0,0,.,0,0,.,266971,139973,.,0,0,0,0,0,.,0
9,8,263779,154397,201220,187885,223824,207595,221634,210519,194676,204190,136242,40456,263779,154397,201220,187885,223824,207595,221634,210519,194676,204190,136242,40456,23.85,23.91,24.02,23.73,23.85,23.79,23.92,23.83,24.01,23.41,NaN,23.15,6291129,3691632,4833908,4458887,5338650,4939515,5301707,5016036,4674755,4780700,3137653,936556,6291129,3691632,4833908,4458887,5338650,4939515,5301707,5016036,4674755,4780700,3137653,936556,665035,375624,511873,466261,558846,507019,548803,518048,470333,482200,307641,78964
15,10,0,.,.,.,.,.,.,.,.,.,.,.,0,.,.,.,.,.,.,.,.,.,.,.,0,.,.,.,.,.,.,.,.,.,.,.,0,.,.,.,.,.,.,.,.,.,.,.,0,.,.,.,.,.,.,.,.,.,.,.,0,.,.,.,.,.,.,.,.,.,.,.


In [0]:
758 * 72

54576

In [0]:
observations.shape


(758, 73)

In [0]:
# Melt each row into 12 months X 6 meaasurements

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
          'August', 'September', 'October', 'November', 'December']

measurements = ["Quantity", "Elec Quantity", "MMBtuPer_Unit", "Tot_MMBtu", 
                "Elec_MMBtu", "Netgen"]

dicts = []
first_data_col = 1 # col zero is plant ID, the thing measured.

for obsv in observations.iterrows():
  row = obsv[1]

  plant_ID = row[0]
  
  measurement_index = 0 # start a twelve month chunk.
  
  for measurement in measurements:
    
    month_index = 0 # Iterate across months of one measurement var.
    
    for month in months:
      data_index = first_data_col + month_index + measurement_index
      row_dict = {'Plant_Id' : plant_ID, 
                   'Month' : month, 
                   'Measurement Type' : measurement, 
                   'Value' : row[data_index] }
    
      dicts = dicts + [row_dict]
      month_index += 1 # move to next month
      
    measurement_index +=12 # move to next twelve measurements
#   print(dicts)

new_df = pd.DataFrame(dicts) 

print(new_df.shape)
new_df.describe

(54576, 4)


<bound method NDFrame.describe of        Plant_Id      Month Measurement Type Value
0             3    January         Quantity     .
1             3   February         Quantity     0
2             3      March         Quantity     .
3             3      April         Quantity     .
4             3        May         Quantity     .
...         ...        ...              ...   ...
54571     99999     August           Netgen     0
54572     99999  September           Netgen     0
54573     99999    October           Netgen     0
54574     99999   November           Netgen     0
54575     99999   December           Netgen     0

[54576 rows x 4 columns]>